In [14]:
import pandas as pd
import numpy as np

class Data_Handler:
    
    def __init__(self):
        self.dflisting = None
        self.dfprice = None
        self.dfmerge = None
        
    def getDatas(self):
        print("Récupération des données")
        listing = pd.read_csv("https://storage.googleapis.com/h3-data/listings_final.csv", sep=";")
        prices = pd.read_csv("https://storage.googleapis.com/h3-data/price_availability.csv", sep=";")
        self.dflisting, self.dfprice = listing, prices
        return "data loaded \nFiles : \n dflisting {} \n dfprice{}".format(listing.shape,prices.shape)
        
    def groupDatas(self):
        print("Fusion des données")
        moyenne = self.dfprice.groupby('listing_id')['local_price'].mean()
        self.dfmerge = pd.merge(moyenne, self.dflisting, on="listing_id")
        print ("Données fusionnées")
        
    def getAndGroup(self):
        self.getDatas()
        self.groupDatas()
        print("Données récupérer et grouper")
        

In [ ]:
data = Data_Handler()
dfmerge = data.getAndGroup()
%time

Récupération des données


In [ ]:
class FeatureRecipe :
    
    def __init__(self, data: DataFrame):
        self.DataFrame = DataFrame
        self.varcat = None
        self.varfloat = None
        self.varint = None
    
    def dropnan(self, threshold: float):
        print("supprimer les colonnes avec NaN")
        nan = self.DataFrame.columns[df.isnull().any()]
        for column in nan:
            print(column, self.DataFrame[column].isnull().sum)
            if self.DataFrame[column].isnull().sum()*100.0/df_shape[0] > 99:
                self.DataFrame.drop(column,1, inplace=True)
        print("Supprimer les colonnes avec 100% de Nan")
        
            if self.DataFrame[column].isnull().sum*100.0/df_shape[0] > self.threshold
        print("Supprimer les colonnes avec un taux de NaN égal ou supérieur ou égale -> thresold")
    def dropuseless(self):
        print("Supprimer les colonnes inutiles")
        self.DataFrame.drop('Unnamed: 0',
                            axis='columns',
                            inplace=True)
    
    def dropduplicate():
        
    
    def separatevariables():
        for column in self.DataFrame.columns:
            if self.DataFrame[column].dtypes == float:
                self.varfloat.append(self.DataFrame[column])
            elif self.DataFrame[column].dtypes == int:
                self.varint.append(self.DataFrame[column])
            else:
                self.varcat.append(self.DataFrame[column])
        
        